# Washington D.C. Crime Statistics

This document analysizes DC Metro crime data collected from:
https://www.kaggle.com/vinchinzu/dc-metro-crime-data.

The data contains 342,867 reports of crime spanning from 2007-2017
    
Neighborhood cluster information was collected from:
https://www.neighborhoodinfodc.org/nclusters/nclusters.html  
    

## Data Setup

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats

In [2]:
dc_crime_data = pd.read_csv('dc_crime_add_vars.csv')
cluster_map = pd.read_csv('cluster_map.csv')

### Overview of Raw Data

In [3]:
dc_crime_data.head()

,Unnamed: 0,X,REPORT_DAT,SHIFT,OFFENSE,METHOD,BLOCK,DISTRICT,PSA,WARD,...,year,month,day,hour,minute,second,EW,NS,quad,crimetype
0,1,1,8/31/2008 8:47:00 PM,EVENING,THEFT/OTHER,OTHERS,3500 - 3599 BLOCK OF R STREET NW,2.0,206.0,2,...,2008,8,31,20,47,0,West,North,Northwest,Non-Violent
1,2,2,9/1/2008 12:45:00 AM,MIDNIGHT,MOTOR VEHICLE THEFT,OTHERS,2000 - 2015 BLOCK OF O STREET NW,2.0,208.0,2,...,2008,9,1,0,45,0,West,North,Northwest,Non-Violent
2,3,3,9/1/2008 3:00:00 AM,MIDNIGHT,THEFT/OTHER,OTHERS,700 - 799 BLOCK OF 11TH STREET NW,1.0,101.0,2,...,2008,9,1,3,0,0,East,North,Northeast,Non-Violent
3,4,4,9/9/2008 7:46:00 AM,DAY,THEFT/OTHER,OTHERS,1700 - 1799 BLOCK OF P STREET NW,2.0,208.0,2,...,2008,9,9,7,46,0,West,North,Northwest,Non-Violent
4,5,5,8/24/2008 8:00:00 PM,EVENING,MOTOR VEHICLE THEFT,OTHERS,500 - 599 BLOCK OF INDIANA AVENUE NW,1.0,102.0,2,...,2008,8,24,20,0,0,East,North,Northeast,Non-Violent


In [4]:
dc_crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342867 entries, 0 to 342866
Data columns (total 32 columns):
Unnamed: 0              342867 non-null int64
X                       342867 non-null int64
REPORT_DAT              342867 non-null object
SHIFT                   342867 non-null object
OFFENSE                 342867 non-null object
METHOD                  342867 non-null object
BLOCK                   342867 non-null object
DISTRICT                342667 non-null float64
PSA                     342616 non-null float64
WARD                    342867 non-null int64
ANC                     342867 non-null object
NEIGHBORHOOD_CLUSTER    338162 non-null object
BLOCK_GROUP             341776 non-null object
CENSUS_TRACT            341776 non-null float64
VOTING_PRECINCT         342783 non-null object
CCN                     342867 non-null int64
START_DATE              342854 non-null object
END_DATE                331216 non-null object
XBLOCK                  342867 non-null floa

The cluster_map data can be used to identify which neighborhoods fall into each neighborhood cluster

In [5]:
cluster_map.head()

,NEIGHBORHOOD_CLUSTER,neighborhoods
0,0,unknown
1,1,"Kalorama Heights, Adams Morgan, Lanier Heights"
2,2,"Columbia Heights, Mt. Pleasant, Pleasant Plain..."
3,3,"Howard University, Le Droit Park, Cardozo/Shaw"
4,4,"Georgetown, Burleith/Hillandale"


In [6]:
pd.set_option('display.max_colwidth', -1)

In [7]:
cluster_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
NEIGHBORHOOD_CLUSTER    40 non-null int64
neighborhoods           40 non-null object
dtypes: int64(1), object(1)
memory usage: 720.0+ bytes


### Clean up the Data

In [8]:
cluster_map.rename(columns={"NEIGHBORHOOD_CLUSTER": "cluster"}, inplace=True)

In [9]:
dc_crime_data.drop(columns=['Unnamed: 0','X','REPORT_DAT','START_DATE','END_DATE', 'minute','second', 'CCN','BLOCK','WARD','optional', 'ANC','BLOCK_GROUP','CENSUS_TRACT'], inplace = True)

In [10]:
dc_crime_data.rename(columns={"NEIGHBORHOOD_CLUSTER": "cluster"}, inplace=True)

In [11]:
dc_crime_data['cluster'].fillna(0, inplace=True)

In [12]:
dc_crime_data['cluster'] = dc_crime_data['cluster'].astype(str)

In [13]:
dc_crime_data['cluster'] = dc_crime_data['cluster'].apply(lambda cluster: cluster.replace("Cluster",""))

In [14]:
dc_crime_data['cluster'] = pd.to_numeric(dc_crime_data['cluster'])

In [15]:
dc_crime_data['count'] = 1
dc_crime_data['violent'] = 0
dc_crime_data['non_violent'] = 0

In [16]:
dc_crime_data.loc[dc_crime_data.crimetype == "Non-Violent", "non_violent"] = 1
dc_crime_data.loc[dc_crime_data.crimetype == "Violent", "violent"] = 1

In [ ]:
dc_crime_data = pd.merge(dc_crime_data,cluster_map,on='cluster')

## Overview Of Data

#### Crimes by Type of Offense

In [ ]:
plt.figure(figsize=(20,5))
plt.title("Crimes by Type of Offense")
sns.countplot(x='OFFENSE',data=dc_crime_data,palette='viridis')
plt.show()
dc_crime_data['OFFENSE'].value_counts()

#### Crimes by Method

In [ ]:
plt.figure(figsize=(20,5))
plt.title("Crimes by Method")
sns.countplot(x='METHOD',data=dc_crime_data,palette='viridis')
plt.show()
dc_crime_data['METHOD'].value_counts()

#### Crimes by Type of Crime

In [ ]:
plt.figure(figsize=(20,5))
plt.title("Crimes by Crime Type")
sns.countplot(x='crimetype',data=dc_crime_data,palette='viridis')
plt.show()
dc_crime_data['crimetype'].value_counts()

#### Crimes by Quadrant Location

In [ ]:
plt.figure(figsize=(20,5))
plt.title("Crimes by Quadrant")
sns.countplot(x='quad',data=dc_crime_data, hue='crimetype')
plt.show()
dc_crime_data['quad'].value_counts()

The distribution of crimes by quadrant shows a majority of crimes occured in the Northeast while practically zero crimes occured in the Southwest.  While population is not represented in the data, it seems likely there is no population in the Southwest.  Looking at a map confirms this suspicion showing very little land in the Southwest quadrent of Washington DC.

It would be likely to find greater amounts of crime in areas with greater amounts of people.  However, population data is not avaiable in this dataset.  While the Northwest incures greater numbers of crime, one might be less likely to suffer from crime if the population is significantly larger.  In other words, in areas which have a greater population, it is more likely that the crime will happen to someone else.  

#### Percentage of crime which is Violent

In [ ]:
quads = dc_crime_data.groupby(['quad']).agg({'violent':'sum', 'non_violent':'sum', 'count':'sum'})
quads['pct_violent'] = quads['violent'] / quads['count']
quads.reset_index(inplace=True)
sns.catplot(x='quad',y='pct_violent',data=quads,kind='bar')
plt.show()
quads.sort_values(by=['pct_violent'], ascending=False)

Areas with larger populations are expected to have larger counts of crime.  For instance, an area with twice as many people would be expected to have twice as much crime.  This means twice as much non-violent crime and twice as much violent crime.  

However, the ratio of violent to non-violent crime is disproportionate between quadrents. This data shows 25% of crime in the Southeast is violent, while only 18% of crime in the Northeast is violent.  This suggests the Southeast could be a more dangerous quadrant despite having less crime overal.  

#### Crime by Neighborhood Clusters

To view a geographical map of the clusters please visit:
https://www.neighborhoodinfodc.org/nclusters/nclusters.html

In [ ]:
plt.figure(figsize=(16,5))
plt.title("dc crimes by neighborhood cluster (2007-2017)")
plt.xlabel('year')
plt.ylabel('crimes')
sns.countplot(x='cluster',data=dc_crime_data,palette='viridis')
plt.show()
cluster_map

## All D.C. Crime by Year

In [ ]:
years = dc_crime_data.groupby(['year']).agg({'violent':'sum', 'non_violent':'sum', 'count':'sum'})
years['pct_violent'] = years['violent'] / years['count']
years.reset_index(inplace=True)
min_year_count = years['count'].min()
max_year_count = years['count'].max()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(years['year'], years['non_violent'], label='non-violent')
plt.plot(years['year'], years['violent'], label='violent')
plt.xlabel('year')
plt.ylabel('crimes')
plt.title("dc crimes (2007-2017)")
plt.legend()
plt.show()
years

The data suggest a dramatic decrease in crime from 2016 to 2017.  Because the data ends in 2017, it seems possible the data only includes 2017 up until the time the data was collected. If data has not been collected for the entire year of 2017 this would explain the dramatic dropoff in number of crimes.

In [ ]:
pd.to_datetime(dc_crime_data[dc_crime_data['year'] == 2017]['date']).sort_values(ascending=False).head(5)

After filtering the data for 2017 and sorting by the date, the last 5 records show there are no records after November 3, 2017.  It would be reasonable to assume the 2017 data is missing after this date.

With two months missing from the 2017 data, 2017 cannont be compared to the other years, and will be removed for the remainder of this analysis when comparing data from year to year.  

In [ ]:
dc_crime_data.drop(dc_crime_data[dc_crime_data.year == 2017].index, inplace=True)
years.drop(years[years.year == 2017].index, inplace=True)

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(years['year'], years['non_violent'], label='non-violent')
plt.plot(years['year'], years['violent'], label='violent')
plt.xlabel('year')
plt.ylabel('crimemax_counts')
plt.title("dc crimes (2007-2016)")
plt.legend()
plt.show()
years

Removing 2017 from the data results in the above graph which now accuratly desribes the changes in crime from year to year

### Linear Regression of Crime 

While the graph above shows both increase and descreases in crime over the years, the trend appears to show non-violent crime is increasing while violent crime is remaining about the sime.  

Linear regression can be used to get a better idea of the overall trend in numbers

In [ ]:
g = sns.lmplot(y='count', 
               x='year',
               data=years, # Data.
               fit_reg=True) # The regression line also includes a 95% confidence envelope.
g.set_ylabels("number of crimes")
g.set_xlabels("year")
g.set(ylim=(3000, 40000))
plt.title('Crimes in DC 2007-2016')

g = sns.lmplot(y='non_violent', 
               x='year',
               data=years, # Data.
               fit_reg=True) # The regression line also includes a 95% confidence envelope.
g.set_ylabels("number of non violent crimes")
g.set_xlabels("year")
g.set(ylim=(3000, 40000))
plt.title('Non-violent Crimes in DC 2007-2016')

g = sns.lmplot(y='violent', 
               x='year',
               data=years, # Data.
               fit_reg=True) # The regression line also includes a 95% confidence envelope.
g.set_ylabels("number of violent crimes")
g.set_xlabels("year")
g.set(ylim=(3000, 40000))
plt.title('Violent Crimes in DC 2007-2016')
plt.show()

While violent crimes look mostly flat,it does appear to be on a slight decline.  Calculating a t-score and p-value will suggest whether the change in crime between the early and later years is no more significant than the change in violent crimes between each year.  Alternativly, the change in crime from the early years to the later years could be significant enough to conclude that violent crime is decreasing significantly.

In [ ]:
early_years_violent = years['violent'][:4]

In [ ]:
later_years_violent = years['violent'][5:]

In [ ]:
early_years_violent_mean = early_years_violent.mean()
later_years_violent_mean = later_years_violent.mean()
diff =  early_years_violent_mean - later_years_violent_mean

early_years_violent_var = np.var(early_years_violent)
later_years_violent_var = np.var(later_years_violent)

early_years_violent_std = np.sqrt(early_years_violent_var)
later_years_violent_std = np.sqrt(later_years_violent_var)

plt.hist(early_years_violent, alpha=0.5, label='2008-2011') 
plt.hist(later_years_violent, alpha=0.5, label='2013-2016') 
plt.legend(loc='upper right') 
plt.title('Violent Crime in Earlier Years vs. Later Years')

# Add a vertical line at the mean.
plt.axvline(early_years_violent_mean, color='blue', linestyle='solid', linewidth=2)
plt.axvline(later_years_violent_mean, color='orange', linestyle='solid', linewidth=2)

# Add a vertical line at one standard deviation above the mean.
plt.axvline(early_years_violent_mean + early_years_violent_std, color='blue', linestyle='dashed', linewidth=2)
plt.axvline(later_years_violent_mean + later_years_violent_std, color='orange', linestyle='dashed', linewidth=2)

# Add a vertical line at one standard deviation below the mean.
plt.axvline(early_years_violent_mean - early_years_violent_std, color='blue', linestyle='dashed', linewidth=2)
plt.axvline(later_years_violent_mean - later_years_violent_std, color='orange', linestyle='dashed', linewidth=2)

plt.show()

print('Mean violent crimes in the earlier years: ',early_years_violent_mean)
print('Mean violent crimes in the later years: ',later_years_violent_mean)
print('Difference between earlier and later year means: ',diff)

print('\nEarly years violent crime variance: ',early_years_violent_var)
print('Later years violent crime variance: ',later_years_violent_var)

print('\nEarly years standard deviation: ',early_years_violent_std)
print('Later years standard deviation: ',later_years_violent_std)

tval,pval=stats.ttest_rel(early_years_violent, later_years_violent)
print("\nt-value: ",tval)
print("p-value: ",pval)

The calculated t-value is 2.4.  This indicateds that the difference between the early years and the later years is about 2.x times the difference between the individual years.

However, the p-value is 0.09, meaning there is a 9% probability that the difference is due to the amount of noise and not because of any significant decrease in violent crime.

Therefore, violent crimes should be considered flat and not declining.

## Top 5 - Crime Data by Neighborhood

In [ ]:
neigh_year = dc_crime_data.groupby(['cluster','neighborhoods', 'year']).agg({'violent':'sum', 'non_violent':'sum', 'count':'sum'})
neigh_year['pct_violent'] = neigh_year['violent'] / neigh_year['count']
neigh_year.rename(columns={"NEIGHBORHOOD_CLUSTER": "cluster", "year": "year", "violent": "total_violent", "non_violent": "total_non_violent", "count": "total_count", "pct_violent": "pct_violent"}, inplace=True)
neigh_year.reset_index(inplace=True)

min_neigh_year_count = (neigh_year['total_violent'].min()*.95)
max_neigh_year_count = (neigh_year['total_non_violent'].max()*1.05)

In [ ]:
neigh_year_avg = neigh_year.groupby(['cluster','neighborhoods']).agg({'total_violent':'mean', 'total_non_violent':'mean', 'total_count':'mean'})
neigh_year_avg['pct_violent'] = neigh_year_avg['total_violent'] / neigh_year_avg['total_non_violent']
neigh_year_avg.rename(columns={"total_violent": "avg_violent", "total_non_violent": "avg_non_violent", "total_count": "avg_count", "pct_violent": "pct_violent"}, inplace=True)
neigh_year_avg.sort_values(by=['avg_count', 'avg_violent'], ascending=False)
neigh_year_avg.reset_index(inplace=True)

neigh_year_avg['2016-2008'] = 0
neigh_year_avg['2016-2008_pct'] = 0
for index, row in neigh_year_avg.iterrows():
    # find the neigh_year records for this cluster
    count08 = (neigh_year[(neigh_year['cluster'] == row['cluster']) & (neigh_year['year'] == 2010)]['total_count']).values
    count016 = (neigh_year[(neigh_year['cluster'] == row['cluster']) & (neigh_year['year'] == 2016)]['total_count']).values
    neigh_year_avg.loc[[index], ['2016-2008']] = count016 - count08
    neigh_year_avg.loc[[index], ['2016-2008_pct']] = count016/count08
    
neigh_year_avg['2016-2008_violent'] = 0
neigh_year_avg['2016-2008_violent_pct'] = 0
for index, row in neigh_year_avg.iterrows():
    # find the neigh_year records for this cluster
    count08 = (neigh_year[(neigh_year['cluster'] == row['cluster']) & (neigh_year['year'] == 2010)]['total_violent']).values
    count016 = (neigh_year[(neigh_year['cluster'] == row['cluster']) & (neigh_year['year'] == 2016)]['total_violent']).values
    neigh_year_avg.loc[[index], ['2016-2008_violent']] = count016 - count08
    neigh_year_avg.loc[[index], ['2016-2008_violent_pct']] = count016/count08

In [ ]:
neigh_year_std = neigh_year.groupby(['cluster','neighborhoods']).agg({'total_violent':'std', 'total_non_violent':'std', 'total_count':'std'})
neigh_year_std.rename(columns={"total_violent": "violent_std", "total_non_violent": "non_violent_std", "total_count": "count_std"}, inplace=True)
neigh_year_std.sort_values(by=['count_std', 'violent_std'], ascending=False)
neigh_year_std.reset_index(inplace=True)


### top 5 neighborhood clusters with most average total crimes over the years

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['avg_count'], ascending=False).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with lowest average total crimes over the years

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['avg_count'], ascending=True).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with most violent crimes over the years

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['avg_violent'], ascending=False).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with highest percentage violent crimes over the years

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['pct_violent'], ascending=False).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with largest std of average crimes over the years

In [ ]:
for index, row in neigh_year_std.sort_values(by=['count_std'], ascending=False).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with smallest std of average crimes over the years

In [ ]:
for index, row in neigh_year_std.sort_values(by=['count_std'], ascending=True).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with biggest increase of all crime

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['2016-2008'], ascending=False).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with biggest decrease of all crime

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['2016-2008'], ascending=True).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with biggest increase of violent crime

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['2016-2008_violent'], ascending=False).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

In [ ]:
neigh_year_avg.sort_values(by=['2016-2008_violent'], ascending=False).head(5)

### top 5 neighborhood clusters with biggest decrease of violent crime

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['2016-2008_violent'], ascending=True).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with biggest increase of crime percentage

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['2016-2008_pct'], ascending=False).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with biggest decrease of crime percentage

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['2016-2008_pct'], ascending=True).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with biggest increase of violent crime percentage

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['2016-2008_violent_pct'], ascending=False).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### top 5 neighborhood clusters with biggest decrease of violent crime percentage

In [ ]:
for index, row in neigh_year_avg.sort_values(by=['2016-2008_violent_pct'], ascending=True).head(5).iterrows():
    print("Cluster: ",row['cluster'], neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods'].head(1).values)

### all clusters

In [ ]:
fig, axes = plt.subplots(40, 1, figsize=(10,128))

rating = 0

for index, row in neigh_year_avg.sort_values(by=['cluster'], ascending=True).head(40).iterrows():
    axes[rating].set_ylim([min_neigh_year_count,max_neigh_year_count])  
    axes[rating].set_xlabel('Year')
    axes[rating].set_ylabel('Number of Crimes')
    title = ("Crime in cluster " + str(row['cluster']) + ": " + (neigh_year[neigh_year['cluster'] == row['cluster']]['neighborhoods']).head(1).values)
    axes[rating].set_title(title)
    axes[rating].plot(neigh_year[neigh_year['cluster'] == row['cluster']]['year'], neigh_year[neigh_year['cluster'] == row['cluster']]['total_non_violent'], label="non-violent")
    axes[rating].plot(neigh_year[neigh_year['cluster'] == row['cluster']]['year'], neigh_year[neigh_year['cluster'] == row['cluster']]['total_violent'], label="violent")
    axes[rating].legend()
    axes[rating].grid(True)
    rating = rating+1

plt.tight_layout()

In [ ]:
neigh_year[neigh_year['cluster'] == 15]